In [5]:
import pandas as pd
# import sys
# print(f"Running Python: {sys.executable}")
# import os
# print(os.getcwd())

def get_data():
    d_train = pd.read_csv("data/train.csv")
    d_test = pd.read_csv("data/test.csv")

    return d_train, d_test


df_train, df_test = get_data()